# Identifying Gender on Twitter

#### by Emma Freeman


## Introduction

In a social media campaign, understanding the makeup of your following is important for delivering the right content to the right consumers. However, information about those consumers is often not publicly available or easy to obtain. The ability to infer information about your following will be an asset to anyone in the social media space.  

Gender is one of the most important determinants in the design of a social media campaign.

To determine the gender of Twitter users, I collected 1.4 million tweets from over 10,000 unique gender-tagged twitter users. I used natural language processing and machine learning to classify and predict the gender of Twitter users based on their tweets.

## Getting the data

My project was inspired by a CrowdFlower project that pulled 20,050 random tweets (10,000 with the word "the" and 10,000 with the word "and" in the tweet), as well as profile information for the tweet authors. They had volunteers categorize each profile/user as "male", “female,” “brand or organization,” or “can’t tell”. They then made a model to predict gender or brand based on the text of the tweets and the color of the profile background, and achieved a 60% predictive accuracy with their model. They also identified the most important features for each category.  https://www.crowdflower.com/using-machine-learning-to-predict-gender/

I downloaded the CrowdFlower dataset as a CSV and loaded it into a Jupyter notebook. I selected only the samples where CrowdFlower's confidence in the gender categorization was above 70% and where the gender is male or female (not brand, NaN or unknown). This left 10,041 rows or users, balanced between males and females. I saved these users' usernames as a list.

I wrote a series of Python scripts that I used with Tweepy and the Twitter API to collect up to 200 tweets for each user in the list. I decided on 200 tweets to try to balance the dataset so that the most prolific tweeters don't overshadow the infrequent tweeters, and for ease of collection while working with the Twitter API limits. 

I saved the tweets directly to a MongoDB collection. I employed a parallel process in which I set up two sets of Twitter API keys to pull tweets faster, in two Jupyter notebooks and on two separate AWS instances. As I pulled the tweets for each user, the username was deleted from the 'incomplete' MongoDB collection and added to either the 'complete' collection, if the script worked and I got their tweets, or the 'failed' collection, if it failed. This resulted in a collection of 1,374,087 tweets.

I decided to only use the tweets that are not retweets. The text of the tweets was encoded in binary, so I couldn't use regex to find non-RTs in the Mongo collection. I wrote a function to find all the tweets in the orginal collection where the user language is set to English, decode the text field, and store the tweet in a new "decoded" Mongo collection. Then I implemented a function to find all tweets that don't start with "RT", and insert them into a new Mongo collection. 

I found that working with the data from a Mongo database was prohibitively slow, for example when pulling tweets to try to work with in Pandas or do NLP, I could only pull about 10,000 at a time. I deleted duplicates from the Mongo collections, and then created a multikey text index to optimize the database performance, but it was still too slow. I decided to go back to the data collection phase and save all the tweets directly to a csv file as I pulled them, and work directly with the csv file.

## Exploratory data analysis and pre-processing

I loaded the tweets from a csv into a Pandas dataframe and dropped duplicates and re-tweets, then cleaned the text (removed URLs, line breaks, and "amp").

Then I added the gender classification from the CrowdFlower dataframe by merging the dataframes, matching on usernames. I pulled the text of the tweets into a numpy array as 'features', and the genders into another numpy array as 'y'. I dropped any that were 'brand' or 'unknown'. This left 901,873 tweets:
  * male: 469,522
  * female: 432,351

For the next step I decided to explore the prevalence of emojis in the tweets. I counted the number of times each emoji appeared in the tweets overall, and then in male vs. female tweets. I found that there are 219,698 total emojis present in the tweet text. 

## Natural Language Processing and Modeling

To identify the top features (words and emojis) for male tweeters and female tweeters, I used LSA (CountVectorizer and then Tfidf, into TruncatedSVD) to vectorize the tweets for each group and get the top features for each.

I wanted to preserve the emojis in the vectorized dataset, which proved rather challenging. I found a text file of all the emojis that exist (https://github.com/emmafreeman/socialmediaparse/blob/master/data/emoji_table.txt) and used this to build a vocabulary list. I then built a dictionary of emojis that indexed each emoji in the tweet text, and replaced each emoji in the tweets with its index value (e.g. 'emoji_1'). This allowed me to vectorize the emojis as if they were text. Later, for visualization purposes, I reversed the dictionary and replaced the emoji words with their original pictorial representations.

I built Logistic Regression and Random Forest models to predict gender from the vectorized text, without doing any hyperparameter tuning.

## Preliminary Results

  * females use about twice as many emojis as males, but the top words used by males and females are very similar
  * the most popular emoji, by far, is the 'face with tears of joy' emoji 😂 
  * both models got an initial predictive accuracy test score of 59%

## Next steps:
  * Improve predictive model accuracy by trying different models and tuning them; could try building a neural network
  * Organize and clean code and files, push to Github
  * Build a script that can take a Twitter user’s handle as input, and output the user’s gender
  * Do some LDA/topic modeling, LDAvis
  * Sentiment analysis on male vs. female tweets